## Imports

In [1]:
import os
from PIL import Image
from glob import glob
import scipy.misc
from random import randrange
import copy
%matplotlib inline

## Helper Functions / Classes

In [44]:
class ImageGen():
    def __init__(self):
        self.base_dir = os.getcwd()
        self.data_dir = os.path.join(self.base_dir, "data")
        self.crop_dir = os.path.join(self.data_dir, "crops")
        self.bg_dir = os.path.join(self.data_dir, "bgs")
        self.gen_dir = os.path.join(self.data_dir, "gen")
        
        # crop images
        self.crop_width = 512
        self.get_crops()
        self.roomba_meter_ratio = 13.39 * 2.54 / 100
        
        # background images
        self.pixels_per_meter = 600
        self.bg_width = 1280
        self.bg_height = 720
        self.get_bgs()
        
    def get_crops(self):
        """
        reads in and resizes cropped images into the self.crop_imgs list
        """
        os.chdir(self.crop_dir)
        self.crop_filenames = sorted(glob("*"))
        self.crop_imgs = [Image.open(filename) for filename in self.crop_filenames]
        self.crop_imgs = [self.resize_img(image, self.crop_width) for image in self.crop_imgs]
        os.chdir(self.base_dir)
        
    def get_bgs(self):
        """
        reads in and resizes background images the self.bg_imgs list
        """
        os.chdir(self.bg_dir)
        self.bg_filenames = sorted(glob("*"))
        self.bg_imgs = [Image.open(filename) for filename in self.bg_filenames]
#         self.bg_imgs = [image.resize((self.bg_width, self.bg_height), resample=Image.BILINEAR)
#                         for image in self.bg_imgs]
        os.chdir(self.base_dir)
        
    def resize_img(self, img, desired_width):
        """
        resizes target img while preserving width 
        
        Args:
            img (PIL Image): the image to be resized
            desired_width: the desired width for the resized image
            
        Returns:
            resized_img (PIL Image): the resized image
        """
        width, height = img.size
        ratio = width / height
        desired_height = int(desired_width / ratio)
        resized_img = img.resize((desired_width, desired_height), resample=Image.BILINEAR)
        return resized_img
    
    def crop_center(self, img, width, height):
        left  = int((img.width / 2) - (width / 2))
        right = left + width
        upper = int((img.height / 2) - (height / 2))
        lower = upper + height
        return img.crop((left, upper, right, lower))
        
    def overlay_imgs(self, fg, bg, x, y, rot, zoom):
        """
        overlays a foreground image onto a background image
        
        Args:
            fg (PIL Image): the foreground image
            bg (PIL Image): the background image
            x (int): the x position of the upper left corner to overlay image
            y (int): the y position of the upper left corner to overlay image
            
        Returns:
            bg_copy (PIL Image): the image of foreground overlayed on background
        """
        fg_copy = copy.deepcopy(fg)
        fg_copy = self.resize_img(fg_copy, int(self.pixels_per_meter * 
                                               self.roomba_meter_ratio * 
                                               zoom))
        fg_copy = fg_copy.rotate(45, expand=True, resample=Image.BILINEAR)
        
        bg_copy = copy.deepcopy(bg)
        bg_copy = bg_copy.rotate(rot, expand=True, resample=Image.BILINEAR)
        bg_copy = self.crop_center(bg_copy,
                                   int(self.bg_width / zoom) + 1, # +1 to fix transparent
                                   int(self.bg_height / zoom) + 1)
        bg_copy = self.resize_img(bg_copy, self.bg_width)
        print(fg_copy.width, fg_copy.height)
        try:
            x = randrange(0, 1280-fg_copy.width)
            y = randrange(0, 720-fg_copy.height)
            bg_copy.paste(fg_copy, (x, y), fg_copy)
            print((x,y), (x+fg_copy.width, y+fg_copy.height))
            return bg_copy
        except:
            return None

In [45]:
imageGen = ImageGen()

In [46]:
fg = imageGen.crop_imgs[1]
bg = imageGen.bg_imgs[2]

In [47]:
from random import randrange

In [50]:
i = 0
while i < 30:
    x = 0
    y = 0
    rot = randrange(0,360)
    height = randrange(150,900) / 300
    zoom = (1280/3600) * 3 / height
    test = imageGen.overlay_imgs(fg, bg, x, y, rot, zoom)
    if test != None:
        test.save("/tmp/test{}.png".format(i))
        i += 1

76 80
(507, 44) (583, 124)
122 129
(1148, 356) (1270, 485)
96 101
(493, 322) (589, 423)
298 316
(274, 19) (572, 335)
131 138
(198, 285) (329, 423)
169 179
(4, 310) (173, 489)
85 90
(54, 608) (139, 698)
104 110
(683, 450) (787, 560)
219 232
(59, 53) (278, 285)
381 404
(526, 285) (907, 689)
231 244
(969, 8) (1200, 252)
309 327
(553, 132) (862, 459)
255 270
(204, 82) (459, 352)
104 110
(1001, 118) (1105, 228)
207 219
(496, 296) (703, 515)
303 321
(202, 190) (505, 511)
159 168
(1, 545) (160, 713)
170 180
(59, 462) (229, 642)
114 120
(109, 323) (223, 443)
211 223
(610, 378) (821, 601)
99 104
(133, 111) (232, 215)
181 191
(500, 242) (681, 433)
142 150
(1085, 408) (1227, 558)
354 375
(67, 303) (421, 678)
101 107
(452, 211) (553, 318)
82 86
(55, 279) (137, 365)
203 215
(780, 10) (983, 225)
101 107
(1060, 496) (1161, 603)
181 191
(949, 161) (1130, 352)
312 330
(359, 51) (671, 381)


In [ ]:
%%timeit
imageGen.resize_img(imageGen.crop_imgs[0], 256);

In [ ]:
test_img = test_img.rotate(randrange(0,360), expand=True)

In [62]:
fg_copy = copy.deepcopy(imageGen.crop_imgs[1])
fg_copy.size

(512, 543)

In [63]:
fg_copy = fg_copy.rotate(45, expand=True)
fg_copy.size

## Get the cropped images

## Get background images

## Generate image

In [ ]:
os.chdir(CROP_DIR)



